# Word rnn
입력('Repeat is the best medicine for) => 모델 => 출력( 'is the best medicine for memory') 


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

## 데이터 전처리

In [2]:
sentence = "Repeat is the best medicine for memory".split()
print(sentence)

['Repeat', 'is', 'the', 'best', 'medicine', 'for', 'memory']


In [4]:
vocab = list(set(sentence))
print(vocab)

['memory', 'best', 'the', 'is', 'medicine', 'Repeat', 'for']


In [6]:
word2index = {tkn: i for i, tkn in enumerate(vocab, 1)}  # 단어에 고유한 정수 부여
word2index['<unk>']=0

print(word2index)

{'memory': 1, 'best': 2, 'the': 3, 'is': 4, 'medicine': 5, 'Repeat': 6, 'for': 7, '<unk>': 0}


In [7]:
print(word2index['memory'])


1


예측을 위해, 정수를 단어로 바꾸는 idx2word 설정

In [9]:
index2word = {v: k for k, v in word2index.items()}
print(index2word)

print(index2word[1])


{1: 'memory', 2: 'best', 3: 'the', 4: 'is', 5: 'medicine', 6: 'Repeat', 7: 'for', 0: '<unk>'}
memory


데이터를 각 정수로 인코딩하면서 입력데이터와 레이블 데이터를 만드는 build_data라는 함수 정의

In [10]:
def build_data(sentence, word2index):
    encoded = [word2index[token] for token in sentence] # 각 문자를 정수로 변환. 
    input_seq, label_seq = encoded[:-1], encoded[1:] # 입력 시퀀스와 레이블 시퀀스를 분리
    input_seq = torch.LongTensor(input_seq).unsqueeze(0) # 배치 차원 추가
    label_seq = torch.LongTensor(label_seq).unsqueeze(0) # 배치 차원 추가
    return input_seq, label_seq

In [11]:
X, Y = build_data(sentence, word2index)

print(X)
print(Y)


tensor([[6, 4, 3, 2, 5, 7]])
tensor([[4, 3, 2, 5, 7, 1]])


## 모델 구현

In [21]:
class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, # 워드 임베딩
                                            embedding_dim=input_size)
        self.rnn_layer = nn.RNN(input_size, hidden_size, # 입력 차원, 은닉 상태의 크기 정의
                                batch_first=batch_first)
        self.linear = nn.Linear(hidden_size, vocab_size) # 출력은 원-핫 벡터의 크기를 가져야함. 또는 단어 집합의 크기만큼 가져야함.

    def forward(self, x):
        # 1. 임베딩 층
        # 크기변화: (배치 크기, 시퀀스 길이) => (배치 크기, 시퀀스 길이, 임베딩 차원)
        output = self.embedding_layer(x)
        # 2. RNN 층
        # 크기변화: (배치 크기, 시퀀스 길이, 임베딩 차원)
        # => output (배치 크기, 시퀀스 길이, 은닉층 크기), hidden (1, 배치 크기, 은닉층 크기)
        output, hidden = self.rnn_layer(output)
        # 3. 최종 출력층
        # 크기변화: (배치 크기, 시퀀스 길이, 은닉층 크기) => (배치 크기, 시퀀스 길이, 단어장 크기)
        output = self.linear(output)
        # 4. view를 통해서 배치 차원 제거
        # 크기변화: (배치 크기, 시퀀스 길이, 단어장 크기) => (배치 크기*시퀀스 길이, 단어장 크기)
        return output.view(-1, output.size(2))

In [22]:
# 하이퍼 파라미터
vocab_size = len(word2index)  # 단어장의 크기는 임베딩 층, 최종 출력층에 사용된다. <unk> 토큰을 크기에 포함한다.
print(vocab_size)
input_size = 5  # 임베딩 된 차원의 크기 및 RNN 층 입력 차원의 크기
hidden_size = 20  # RNN의 은닉층 크기|

8


In [23]:
model = Net(vocab_size, input_size, hidden_size, batch_first=True)

In [24]:
# 손실함수 정의
loss_function = nn.CrossEntropyLoss() # 소프트맥스 함수 포함이며 실제값은 원-핫 인코딩 안 해도 됨.
# 옵티마이저 정의
optimizer = optim.Adam(params=model.parameters())

In [26]:
output = model(X)
print(output)
print(output.shape)


tensor([[-2.4867e-01,  4.1195e-01, -1.4810e-01, -1.2339e-01, -9.4468e-03,
         -3.9808e-01, -9.4959e-02, -1.7064e-01],
        [-3.1736e-01,  1.1791e-01, -9.4132e-02, -1.7043e-01, -4.3318e-01,
         -1.7848e-01, -5.6004e-02,  1.9365e-01],
        [-1.9862e-02,  2.2149e-03, -1.7310e-01,  1.3931e-02,  4.4104e-04,
         -3.6883e-01, -5.2622e-02, -1.0300e-01],
        [-2.0751e-01,  9.6470e-02, -2.2568e-01, -1.8610e-01, -1.7095e-01,
         -4.9345e-01, -1.4436e-01,  1.8445e-02],
        [-8.9860e-02,  3.0934e-01, -1.4511e-01, -2.4488e-01, -3.7997e-01,
          3.2222e-01,  3.0310e-01, -3.6012e-02],
        [-1.6931e-01,  4.0180e-01, -3.9625e-02, -2.0479e-02, -2.1412e-01,
         -1.3897e-01, -1.3002e-01,  6.6080e-02]], grad_fn=<ViewBackward>)
torch.Size([6, 8])


예측값의 크기는 (6, 8) =>  (시퀀스의 길이, 은닉층의 크기)

잘 예측했는지 확인하는 데이터 => 정수로 바꾸는 함수 정의

In [29]:
decode = lambda y: [index2word.get(x) for x in y]

In [30]:
for step in range(201):
    # 경사 초기화
    optimizer.zero_grad()
    # 순방향 전파
    output = model(X)
    # 손실값 계산
    loss = loss_function(output, Y.view(-1))
    # 역방향 전파
    loss.backward()
    # 매개변수 업데이트
    optimizer.step()
    # 기록
    if step % 40 == 0:
        print("[{:02d}/201] {:.4f} ".format(step+1, loss))
        pred = output.softmax(-1).argmax(-1).tolist()
        print(" ".join(["Repeat"] + decode(pred)))
        print()

[01/201] 2.0970 
Repeat memory for the memory medicine memory

[41/201] 1.4392 
Repeat memory the best medicine for memory

[81/201] 0.7830 
Repeat is the best medicine for memory

[121/201] 0.3804 
Repeat is the best medicine for memory

[161/201] 0.2045 
Repeat is the best medicine for memory

[201/201] 0.1259 
Repeat is the best medicine for memory

